In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ksource as ks
import mcpl

# Open tally

### Input cell:
It is recommiended to complete template in a duplicated cell

In [ ]:
outputfile = "filename" # Simulation output file
tallyname = "tally"     # Tally name
spectrum = "spectrum"   # Energy decay spectrum (only for activation tallies)

tally = ks.T4Tally(outputfile, tallyname)

# Create plots

In [ ]:
cell = None # Index of cell of non-plotted variable. None to average all cells.

[fig, [scores,errs]] = tally.plot2D(['z', 'x'], cell=cell)

# Save as particle list

In [ ]:
mcplname = None # Photon list file name (None: create default name)

mcplname = tally.save_tracks(tracksfile)

# Create KSource

In [ ]:
# Open particle list

mcplfile = mcplfile
readformat = "mcpl"
pt = "p"
trasl = None
rot = None
x2z = False
set_params = True
plist = ks.PList(mcplfile, readformat, pt=pt, trasl=trasl, rot=rot, switch_x2z=x2z, set_params=set_params)

# Create Geometry

trasl = None
rot = None
volparams = [tally.grids[0][0], tally.grids[0][-1],
             tally.grids[1][0], tally.grids[1][-1],
             tally.grids[2][0], tally.grids[2][-1]]
metric = ks.Geometry([ks.Lethargy(10), ks.Vol(*volparams), ks.Isotrop()], trasl=trasl, rot=rot)

# Create KSource

J = 1 # Activation source intensity [1/s]
s = ks.KSource(plist, metric, bw="silv", J=J)

# Evaluate statistics

In [ ]:
# With MCPL

file = s.plist.filename
mcpl.dump_stats(mcpl.collect_stats(file))
mcpl.plot_stats(file)

In [ ]:
# With ks.Stats

# Select variables ranges (box)
maskfun = ks.Box(None, None)

# Weighting function
fact_dosim = ks.H10(pt=plist.pt)
weightfun = lambda parts: fact_dosim(parts[:,0])

parts,ws = plist.get()
stats = ks.Stats(parts, ws, weightfun=weightfun, maskfun=maskfun)

N,I,err = stats.mean_weight(steps=100)
plt.show()
N,mn,err = stats.mean(var=1, steps=100)
plt.show()
N,mn,err = stats.std(var=1, steps=100)
plt.show()

# Optimize BW

In [ ]:
# Silverman method

s.bw_method = "silv"
N = 1E4
s.fit(N)
s.scaling[0] = 0 # Discrete energies
s.scaling[4] = s.scaling[5] = s.scaling[6] = np.inf # Isotropic emission

In [ ]:
# Max Likelihood Cross Validation method

s.bw_method = "mlcv"
N = 1E4
s.fit(N)
s.scaling[0] = 0 # Discrete energies
s.scaling[4] = s.scaling[5] = s.scaling[6] = np.inf # Isotropic emission

In [ ]:
# Metodo de K Nearest Neighbours

s.bw_method = "knn"
N = -1
s.fit(N)
s.scaling[:,0] = 0 # Discrete energies
s.scaling[:,4] = s.scaling[:,5] = s.scaling[:,6] = np.inf # Isotropic emission

print("Mean bw:")
print(s.bw.mean(axis=0))

# Save KSource

In [ ]:
xmlfilename = None # XML source parameters file name
bwfilename = None  # BW file name. Only for adaptive BW

xmlfilename = s.save(xmlfilename, bwfilename)